## 24 July 2018
-- Laurin Gray

Separate magnitude data for each source in the red candidate catalog into individual files & convert it to flux density (Janskys) so that it can be run through the DESK SED fitter.

Data is from the catalog of Spitzer sources of Khan et al. (2015), matched with sources from Whitelock et al. (2013) in CasJobs

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import gaussian_kde
import csv
import pathlib

In [2]:
def source_sep(catalog, folder):
    """
    Each source needs to be in its own csv file with wavelengths (in microns, not JHK) in the first column 
    and corresponding flux densities (converted from magnitudes to Janskys) in the second column 
    to run through the Dusty Evolved Star Kit (DESK) for SED fitting.
    
    This function goes through each row in a catalog, finds the magnitudes for that source, converts the magnitude
    to a flux density in units of Janskys, creates a csv file for the source (named the same as the ID), 
    and writes the wavelength & the flux density into the columns of the file.
    
    Note that you must create the folder you wish to put the contents of each catalog into 
    before running the function, and make sure that filename in the function definition points where you want it to.
    Additionally, you must include a / at the end of the folder name when you create the folder variable.
    
    catalog is a pandas dataframe object, and folder is a string.
    
    The flux zero points are from:
    http://www.astro.sunysb.edu/aevans/PHY523/classnotes523/useful-definitions-pp.pdf (for JHK)
    IRAC Instrument handbook (for 3.6, 4.5, 5.8, & 8.0)
    MIPS Instrument handbook (for 24)
    
    Call example:
        catalog = pd.read_csv('/Users/lgray/Documents/Phot_data/Red_Cand_Catalogs/17July2018_LG_RedCand_10.csv')
        folder = 'in_ten/'
        source_sep(catalog, folder)
    """
    
    ID = catalog.ID.values
    col_list = ['Jmag', 'Hmag', 'Kmag', 'k36mag', 'k45mag', 'k58mag', 'k80mag', 'k24mag']
    wavelengths = [1.22, 1.63, 2.19, 3.6, 4.5, 5.8, 8.0, 24]
    f0 = [1570, 1020, 636, 280.9, 179.7, 115.0, 64.9, 7.17] # flux zero points for each wavelength in Jy
    
    c=0
    for index, row in catalog.iterrows():
        mags = []
        for col in col_list:
            mags.append(row[col])
        
        # Convert mags to flux density (fd) in Janskys
        # fd = f0 * 10^(-mag/2.5)
        flux_dens = []
        j = 0
        for i in mags:
            exp = i/-2.5
            fd = f0[j]*np.power(10, exp)
            j = j+1
            flux_dens.append(fd)
        
        # Save to file
        filename = '/Users/lgray/Documents/Phot_data/SED_Fit_Sources/24July2018/'+folder+str(ID[c])+'.csv'
        c = c+1
    
        f = open(filename, 'w')
        writer = csv.writer(f)
    
        k = 0
        for w in wavelengths:
            writer.writerow([w, flux_dens[k]])
            k = k+1

In [7]:
catalog = pd.read_csv('/Users/lgray/Documents/Phot_data/Red_Cand_Catalogs/24July2018/24July2018_LG_RedCand_8.csv')
folder = 'in_eight/'
source_sep(catalog, folder)